## Final project

In [1]:
import facebook_scraper as fs
import pandas as pd
import datetime
import numpy as np
import os

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FANPAGE_LINK = "sieuanhhungMarvel"
current_dir = os.getcwd()
FOLDER_PATH = current_dir + "/Data/"
print(FOLDER_PATH)

/home/vnn04/Documents/facebook_scraper_page/Data/


#### Nhập tài khoản và mật khẩu facebook

In [3]:
user_email = "lovecrushntn@gmail.com"
user_password = "matkhauday123"

In [4]:
day_start = datetime.datetime(2023, 11, 14, 0, 0, 0)
day_end = datetime.datetime(2023, 9, 14, 0, 0, 0)

#### Lấy thông tin của các bài viết trong 2 tháng vừa qua (từ 11/9/2023 - 11/11/2023)

In [13]:
post_list = []

for post in fs.get_posts(FANPAGE_LINK,
                    options={"comments": False, "reactions": False, "allow_extra_requests": False, "fields": "id,message,created_time,full_picture"},
                    extra_info=False, pages=1000, credentials=(user_email, user_password)):
    if post['time'] > day_start:
        continue
    else:
        if post['time'] <= day_end:
            break
        print(post)
        post_list.append(post)


{'post_id': '731679755670029', 'text': 'Một nhà hiền triết quê Thái Bình từng nói', 'post_text': 'Một nhà hiền triết quê Thái Bình từng nói', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 13, 23, 42, 17), 'timestamp': 1699893737, 'image': None, 'image_lowquality': 'https://scontent.fhan4-2.fna.fbcdn.net/v/t39.30808-6/401652984_731679665670038_524313091640589685_n.jpg?stp=cp0_dst-jpg_e15_q65_s320x320&_nc_cat=108&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=sfyTH9X72zwAX_ctQgf&_nc_ht=scontent.fhan4-2.fna&oh=00_AfCivSMIFYMvAQUE4hXFh9XKiJIR-igUPL1I0RByc7aOWg&oe=655CBC57', 'images': None, 'images_description': None, 'images_lowquality': ['https://scontent.fhan4-2.fna.fbcdn.net/v/t39.30808-6/401652984_731679665670038_524313091640589685_n.jpg?stp=cp0_dst-jpg_e15_q65_s320x320&_nc_cat=108&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=sfyTH9X72zwAX_ctQgf&_nc_ht=scontent.fhan4-2.fna&oh=00_AfCivSMIFYMvAQUE4hXFh9XKiJIR-igUPL1I0RByc7aOWg&oe=655CBC57'], 'images_lo

KeyboardInterrupt: 

In [14]:
print(len(post_list))

78


In [17]:
df = pd.DataFrame(post_list)
df.columns

Index(['post_id', 'text', 'post_text', 'shared_text', 'original_text', 'time',
       'timestamp', 'image', 'image_lowquality', 'images',
       'images_description', 'images_lowquality',
       'images_lowquality_description', 'video', 'video_duration_seconds',
       'video_height', 'video_id', 'video_quality', 'video_size_MB',
       'video_thumbnail', 'video_watches', 'video_width', 'likes', 'comments',
       'shares', 'post_url', 'link', 'links', 'user_id', 'username',
       'user_url', 'is_live', 'factcheck', 'shared_post_id', 'shared_time',
       'shared_user_id', 'shared_username', 'shared_post_url', 'available',
       'comments_full', 'reactors', 'w3_fb_url', 'reactions', 'reaction_count',
       'with', 'page_id', 'sharers', 'image_id', 'image_ids', 'was_live',
       'header'],
      dtype='object')

#### Lưu danh sách thông tin cơ bản của các post vào file sieuanhhungMarvel.csv

In [42]:
import csv

path = FOLDER_PATH + FANPAGE_LINK + ".csv"

with open(path, 'w', newline='', encoding='utf-8') as csv_file:
    # Tạo đối tượng csv.writer
    csv_writer = csv.writer(csv_file)

    # Ghi header
    header = post_list[0].keys()
    csv_writer.writerow(header)

    # Ghi dữ liệu
    for row in post_list:
        csv_writer.writerow(row.values())


#### Đọc post_id từ file sieuanhhungMarvel.csv

In [4]:
path = FOLDER_PATH + FANPAGE_LINK + ".csv"
df_post = pd.read_csv(path)

df_post_id = df_post['post_id']
df_post_id.head(10)

0    730004149170923
1    729861169185221
2    729701689201169
3    729692509202087
4    729666115871393
5    729398519231486
6    729351225902882
7    729292349242103
8    729158322588839
9    728884865949518
Name: post_id, dtype: int64

#### Lấy user_id của những người thả reaction

In [33]:
post_id = '728409392663732'
reactor_id = []
for reactor in fs.get_reactors(post_id=post_id, cookies='cookies.txt'):
    print(reactor)
    reactor_id.append(reactor)

In [34]:
a = reactor_id[1]
print(a)

IndexError: list index out of range

In [31]:
print(a['link'])

https://facebook.com/profile.php?id=100095122470934&eav=AfaJYf7oYWzs_aN5rutrgqOoTcamiLxFqL45iQEFDF7_-bWnrpKMbOy0OWrvVxYupxk&fref=pb&paipv=0


In [45]:
text = fs.get_profile('troi2004day')

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/facebook_scraper/facebook_scraper.py:908: UserWarning: Facebook served mbasic/noscript content unexpectedly on https://m.facebook.com/timeline/app_collection/?collection_token=100013048213229%3A103382489711013%3A33&paipv=0&eav=AfZUNq8OlQ8UPqfk-gAfPO4yB9FGAhgKsVkQ3VZ7BYR-olY5hsbljIjLYNyFr46qzVU&_rdr
  warnings.warn(


{'top_post': {'post_id': '1455738051537788', 'text': 'Hãy nhớ rằng: "Nếu thực sự muốn thì người ta sẽ tìm cách, còn nếu không muốn thì người ta sẽ tìm lý do."', 'post_text': 'Hãy nhớ rằng: "Nếu thực sự muốn thì người ta sẽ tìm cách, còn nếu không muốn thì người ta sẽ tìm lý do."', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2022, 7, 14, 6, 33), 'timestamp': None, 'image': None, 'image_lowquality': 'https://scontent.fhan4-3.fna.fbcdn.net/v/t39.30808-6/292428127_5485895144829908_7923343886320211679_n.png?stp=dst-png_s320x320&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=UYEqCdrxttUAX9khrtM&_nc_ht=scontent.fhan4-3.fna&oh=00_AfDnyzA7ZFjc0H4l3fHFlXhWiTclnmQYkb-VY5YwJ4lu2A&oe=655653A8', 'images': [], 'images_description': [], 'images_lowquality': ['https://scontent.fhan4-3.fna.fbcdn.net/v/t39.30808-6/292428127_5485895144829908_7923343886320211679_n.png?stp=dst-png_s320x320&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=UYEqCdrxttUAX9khrtM&_n

In [46]:
text['Thông tin cơ bản']

'17 tháng 10, 2003\nNgày sinh\nNam\nGiới tính'

In [47]:
split_text = text['Thông tin cơ bản'].split('\n')
ngaysinh = split_text[0]
gioitinh = split_text[2]

print(ngaysinh, '\n', gioitinh)

17 tháng 10, 2003 
 Nam
